<h1 style="color:red"><b>Disclaimer of Liability</b></h1>

**The material and information contained on this website is for general information, reference, and self-learning purposes only. You should not rely upon the material or information on the website as a basis for making any academic, business, legal or any other decisions. You should not copy any material or information on the website into any of your academic, business, legal or any other non-private usages. ZHANG Wengyu will not be responsible for any consequences due to your violations.**


Whilst ZHANG Wengyu endeavours to keep the information up to date and correct, ZHANG Wengyu makes no representations or warranties of any kind, express or implied about the completeness, accuracy, reliability, suitability or availability with respect to the website or the information, products, services or related graphics contained on the website for any purpose. Any reliance you place on such material is therefore strictly at your own risk.


ZHANG Wengyu will not be liable for any false, inaccurate, inappropriate or incomplete information presented on the website.


Although every effort is made to keep the website up and running smoothly, due to the nature of the Internet and the technology involved, ZHANG Wengyu takes no responsibility for and will not be liable for the website being temporarily unavailable due to technical issues (or otherwise) beyond its control or for any loss or damage suffered as a result of the use of or access to, or inability to use or access this website whatsoever.


Certain links in this website will lead to websites which are not under the control of ZHANG Wengyu. When you activate these you will leave ZHANG Wengyu's  website. ZHANG Wengyu has no control over and accepts no liability in respect of materials, products or services available on any website which is not under the control of ZHANG Wengyu.


To the extent not prohibited by law, in no circumstances shall ZHANG Wengyu be liable to you or any other third parties for any loss or damage (including, without limitation, damage for loss of business or loss of profits) arising directly or indirectly from your use of or inability to use, this site or any of the material contained in it.

# Advancing House Price Predictions: Data to Model Insights
Group Name: Pangolin 

JIANG Yiyang

ZHANG Wengyu

Deparment of Computing, The Hong Kong Polytechnic University

Nov. 2023

References:
- erikbruin, "House prices: Lasso, XGBoost, and a detailed EDA," *Kaggle.com*, May 10, 2018. https://www.kaggle.com/code/erikbruin/house-prices-lasso-xgboost-and-a-detailed-eda.
- lavanyashukla01, "How I made top 0.3% on a Kaggle competition," *Kaggle.com*, Jun. 09, 2019. https://www.kaggle.com/code/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition.
‌

In [ ]:
import numpy as np 
import pandas as pd 
import os
from datetime import datetime
from scipy.stats import yeojohnson
from scipy.stats import skew 
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.model_selection import train_test_split

from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from mlxtend.regressor import StackingCVRegressor
import pandas as pd
import numpy as np
from scipy.special import boxcox1p
from scipy.stats import skew, boxcox_normmax
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.shape, test.shape


# Feature Engineering

In [ ]:
train_ID = train['Id']
test_ID = test['Id']
# Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop(['Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)

# Deleting outliers
train = train[train.GrLivArea < 4500]
train.reset_index(drop=True, inplace=True)

# We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["SalePrice"] = np.log1p(train["SalePrice"])
y = train.SalePrice.reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

features = pd.concat([train_features, test_features]).reset_index(drop=True)
print(features.shape)
# Some of the non-numeric predictors are stored as numbers; we convert them into strings 
features['MSSubClass'] = features['MSSubClass'].apply(str)
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

features['Functional'] = features['Functional'].fillna('Normal')
features['Electrical'] = features['Electrical'].fillna("SBrkr")
features['KitchenQual'] = features['KitchenQual'].fillna("TA")
features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0])
features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

features["PoolQC"] = features["PoolQC"].fillna("None")

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    features[col] = features[col].fillna(0)
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    features[col] = features[col].fillna('None')
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('None')

features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

objects = []
for i in features.columns:
    if features[i].dtype == object:
        objects.append(i)

features.update(features[objects].fillna('None'))

features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Filling in the rest of the NA's
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics.append(i)
features.update(features[numerics].fillna(0))

In [ ]:
import seaborn as sns
from scipy.stats import norm
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics2.append(i)

skew_features = features[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

# for i in skew_index:
#     features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))

for i in skew_index:
    features[i] = np.log1p(features[i])

# for i in skew_index:
#     features[i], _ = yeojohnson(features[i]+1) 


sns.distplot(features['LotArea'], kde = True, hist=True, fit = norm)
plt.show()

In [ ]:
features = features.drop(['Utilities', 'Street', 'PoolQC',], axis=1)

features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']
features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']

features['Total_sqr_footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                 features['1stFlrSF'] + features['2ndFlrSF'])

features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                               features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))

features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
                              features['EnclosedPorch'] + features['ScreenPorch'] +
                              features['WoodDeckSF'])

# simplified features
features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
features['has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

print(features.shape)
final_features = pd.get_dummies(features).reset_index(drop=True)
print(final_features.shape)

X = final_features.iloc[:len(y), :]
X_sub = final_features.iloc[len(X):, :]

print('X', X.shape, 'y', y.shape, 'X_sub', X_sub.shape)

outliers = [30, 88, 462, 631, 1322]
X = X.drop(X.index[outliers])
y = y.drop(y.index[outliers])

overfit = []
for i in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 99.94:
        overfit.append(i)

overfit = list(overfit)
overfit.append('MSZoning_C (all)')

X = X.drop(overfit, axis=1).copy()
X_sub = X_sub.drop(overfit, axis=1).copy()

print('X', X.shape, 'y', y.shape, 'X_sub', X_sub.shape)
              


# Model Build

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

# rmsle
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


# build our model scoring function
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y,
                                    scoring="neg_mean_squared_error",
                                    cv=kfolds))
    return (rmse)

# setup models    
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=kfolds,))

lasso = make_pipeline(RobustScaler(),
                      LassoCV(max_iter=int(1e7), alphas=alphas2,
                              random_state=42, cv=kfolds))

elasticnet = make_pipeline(RobustScaler(),
                           ElasticNetCV(max_iter=int(1e7), alphas=e_alphas,
                                        cv=kfolds, random_state=42, l1_ratio=e_l1ratio))
                                        
svr = make_pipeline(RobustScaler(),
                      SVR(C= 20, epsilon= 0.008, gamma=0.0003,))


gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
                                   

lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )
                                       

xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006, random_state=42)

# stack
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet,
                                            gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [ ]:
print('TEST score on CV')

score = cv_rmse(ridge)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(svr)
print("SVR score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lightgbm)
print("Lightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(gbr)
print("GradientBoosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(xgboost)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )



In [ ]:
print('START Fit')
print(datetime.now(), 'StackingCVRegressor')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))
print(datetime.now(), 'elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)
print(datetime.now(), 'lasso')
lasso_model_full_data = lasso.fit(X, y)
print(datetime.now(), 'ridge')
ridge_model_full_data = ridge.fit(X, y)
print(datetime.now(), 'svr')
svr_model_full_data = svr.fit(X, y)
print(datetime.now(), 'GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)
print(datetime.now(), 'xgboost')
xgb_model_full_data = xgboost.fit(X, y)
print(datetime.now(), 'lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

# Submission 

In [ ]:
def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) + \
            (0.05 * lasso_model_full_data.predict(X)) + \
            (0.1 * ridge_model_full_data.predict(X)) + \
            (0.1 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.15 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.3 * stack_gen_model.predict(np.array(X))))

submission = pd.read_csv("sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(blend_models_predict(np.array(X_sub))))
q1 = submission['SalePrice'].quantile(0.0042)
q2 = submission['SalePrice'].quantile(0.99)

submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x > q1 else x*0.77)
submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x < q2 else x*1.1)

submission.to_csv("./House_price_submission_blend_v7.csv", index=False)


In [ ]:
def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) + \
            (0.05 * lasso_model_full_data.predict(X)) + \
            (0.1 * ridge_model_full_data.predict(X)) + \
            (0.1 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.15 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.3 * stack_gen_model.predict(np.array(X))))

print(rmsle(y, blend_models_predict(X)))

submission = pd.read_csv("sample_submission.csv")

sub_1 = pd.read_csv('House_price_submission_blend_v6.csv')
sub_2 = pd.read_csv('House_price_submission_blend_v7.csv')
sub_3 = pd.read_csv('House_price_submission_blend_v8.csv')

submission.iloc[:,1] = np.floor((0.7 * sub_1.iloc[:,1]) + 
                                (0.2 * sub_2.iloc[:,1]) + 
                                (0.1 * sub_3.iloc[:,1]) 
                                )


submission.to_csv("House_price_submission_blend2_v10.csv", index=False)

# Further Exploration

## 1. Multiple Layer Perceptron (MLP)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import os, sys, math
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import torch.optim as optim

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Data preprocessing
train_df=pd.read_csv(".train.csv")

text_df=pd.read_csv(".test.csv")

# Split the data into training and validation sets
train_data, valid_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Separate the target variable from training and validation data
y_train = train_data.pop('SalePrice')/100000
y_valid = valid_data.pop('SalePrice')/100000

# Identify numeric and categorical columns
numeric_features = train_data.select_dtypes(include=['float64', 'int64']).columns.drop(['Id']).tolist()
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()

# Build transformers for the numeric and categorical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess the data
X_train = preprocessor.fit_transform(train_data)
X_valid = preprocessor.transform(valid_data)
X_test = preprocessor.transform(text_df)

print(X_train.shape, X_valid.shape, X_test.shape)

X_train = X_train.toarray()
X_valid = X_valid.toarray()
X_test  = X_test.toarray()
y_valid = y_valid.to_numpy()



# MLP Model Build
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        eps = 1e-6
        loss = torch.sqrt(criterion(x, y) + eps)
        return loss

class MLP(nn.Module):
    def __init__(self, indim, blockNum = 2, scale=4, dropout=0.2):
        super(MLP, self).__init__()
        self.indim = indim
        self.dim = self.indim * scale
        self.infc = nn.Sequential(
            nn.Linear(self.indim, self.dim),
            nn.ReLU(inplace=True)
        )
        
        def create_block():
            block = nn.Sequential(
                nn.Linear(self.dim, self.dim),
                nn.BatchNorm1d(self.dim),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),

                nn.Linear(self.dim, self.dim),
                nn.BatchNorm1d(self.dim),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),

                nn.Linear(self.dim, self.dim),
                nn.BatchNorm1d(self.dim)
            )
            return block

        self.blocks = nn.ModuleList([create_block() for _ in range(blockNum)])
        self.out = nn.Linear(self.dim, 1)
    
    def forward(self, x):
        x = x.to(torch.float32)
        x = self.infc(x)
        for block in self.blocks:
            x = block(x) + x  # Add the output of the block to the input (residual connection)
            x = F.relu(x)     # Apply ReLU activation function
        
        out = self.out(x)   

        return out 
    
model = MLP(301).cuda()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def adjust_learning_rate(optimizer, epoch, lr, total_epochs, schedule=None, if_cos=True):
    """Decay the learning rate based on schedule"""
    if if_cos:  # cosine lr schedule
        lr *= 0.5 * (1.0 + math.cos(math.pi * epoch / total_epochs))
    else:  # stepwise lr schedule
        for milestone in schedule:
            lr *= 0.1 if epoch >= milestone else 1.0
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
        

def testModel():
    total_val_loss = 0
    for X_batch_val, y_batch_val in valid_loader:
        # Move data to device
        X_batch_val = X_batch_val.to(device)
        y_batch_val = y_batch_val.to(device)
        
        model.eval()
        
        # Forward pass
        x_val = X_batch_val
        
        predictions_val = model(x_val)
        
        # Compute the loss
        loss_test = testLoss(predictions_val, y_batch_val.unsqueeze(1))
        
        total_val_loss += loss_test.item()
    return total_val_loss/len(valid_loader)
           

# Loss function
criterion = RMSELoss().cuda()

testLoss = RMSELoss().cuda()

lr = 0.001
# # Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

batch_size = 32

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32)

valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Training Loop
num_epochs = 300
train_losses = []
val_losses = []
best_val_loss = 100000
best_train_loss = 100000
best_model = None
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    
    for X_batch, y_batch in train_loader:
        
        if epoch >= 100:
            adjust_learning_rate(optimizer, epoch, lr, num_epochs-100)
        
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        model.train()
        
        # Forward pass
        x = X_batch
        
        
        predictions = model(x)
        
        # Compute the loss
        loss = criterion(predictions, y_batch.unsqueeze(1))
        
        # Zero out any gradients
        optimizer.zero_grad()
        
        # Backward pass
        loss.backward()
        
        # Update the parameters
        optimizer.step()
        
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    if avg_loss < best_train_loss:
        best_train_loss = avg_loss
    
    test_loss = testModel()
    
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        best_model = model
    
    val_losses.append(test_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.6f}, Test Loss: {test_loss:.6f}")
    
plt.figure(figsize = (12,5))
plt.plot(range (0, num_epochs), train_losses, label='Train')
plt.plot(range (0, num_epochs), val_losses, label='Vaild')
plt.legend() 

# Test the mode
best_model.eval()  # Set the model to evaluation mode
test_predictions = best_model(X_test_tensor.cuda())

test_predictions*=100000

test_arr = test_predictions.squeeze(-1).cpu().detach().numpy()

text_df=pd.read_csv("./test.csv")
submission_transformer = pd.DataFrame({'Id': text_df['Id'], 'SalePrice': test_arr})
submission_transformer.to_csv('./MLP_submssion.csv', index=False)


## 2. Transformer

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import tensorflow as tf
from tensorflow.keras.layers import Dense, MultiHeadAttention, Flatten, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Load the provided datasets again
train_data_provided = pd.read_csv("train.csv")
test_data_provided = pd.read_csv("test.csv")

# Remove the 'Id' column as it's not a feature for prediction
train_ids = train_data_provided.pop('Id')
test_ids = test_data_provided.pop('Id')


# Split the data into training and validation sets
train_data, valid_data = train_test_split(train_data_provided, test_size=0.2, random_state=42)

# Separate the target variable from training and validation data
y_train = train_data.pop('SalePrice')
y_valid = valid_data.pop('SalePrice')

# Identify numeric and categorical columns
numeric_features = train_data.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()

# Build transformers for the numeric and categorical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess the data
X_train = preprocessor.fit_transform(train_data)
X_valid = preprocessor.transform(valid_data)
X_test_provided = preprocessor.transform(test_data_provided)

X_train.shape, X_valid.shape, X_test_provided.shape


# Convert the sparse matrices to dense numpy arrays
X_train = X_train.toarray()
X_valid = X_valid.toarray()
X_test_provided  = X_test_provided.toarray()

def RMSE(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))


def simpler_transformer_block(inputs, node_size):
    # Expand the dimension to simulate a sequence
    expanded_inputs = tf.keras.layers.Reshape((-1, 1))(inputs)
    
    # Multi-head self-attention with fewer heads
    attn_output = MultiHeadAttention(num_heads=2, key_dim=node_size)(expanded_inputs, expanded_inputs)
    return attn_output

def build_simpler_transformer_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = Dense(32, kernel_initializer='he_normal')(inputs)
    x = LeakyReLU()(x)  # Use LeakyReLU activation
    
    # Apply simpler Transformer block
    x = simpler_transformer_block(x, node_size=32)
    
    x = Flatten()(x)
    x = Dense(16, kernel_initializer='he_normal')(x)
    x = LeakyReLU()(x)  # Use LeakyReLU activation
    outputs = Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                  loss=RMSE,  # Use RMSE as loss function
                  metrics=[RMSE])  # Use RMSE as a metric as well
    return model

# Initialize the simpler transformer model
simpler_transformer_model = build_simpler_transformer_model(X_train.shape[1])

# Just a summary to visualize the architecture
simpler_transformer_model.summary()

# Define early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.009)

# Train the simpler transformer model
history = simpler_transformer_model.fit(X_train, y_train, epochs=400, 
                                        validation_data=(X_valid, y_valid), 
                                        batch_size=32, verbose=1,
                                        callbacks=[early_stopping, reduce_lr])

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Evaluate the simpler transformer model on the validation set
val_loss, val_mse = simpler_transformer_model.evaluate(X_valid, y_valid)
print("Validation MSE:", val_mse)

# Predict on the test set
predictions_simpler_transformer = simpler_transformer_model.predict(X_test_provided)

# Create submission DataFrame for the simpler transformer model
submission_simpler_transformer = pd.DataFrame({'Id': test_ids, 'SalePrice': predictions_simpler_transformer.squeeze()})
submission_simpler_transformer.to_csv('Transformer_submission.csv', index=False)
